Данные в ML обычно представляются в виде таблицы (матрицы), где:

Строки — это объекты (samples, instances).

Столбцы — это признаки (features). Обозначаются как X.

Целевая переменная — это то, что мы предсказываем. Обозначается как y (только в обучении с учителем).

Нельзя проверять качество модели на тех же данных, на которых она училась (иначе модель просто "запомнит" ответы).
Данные делят на:

Train (Обучающая выборка): ~70-80%. На ней модель находит закономерности.

Test (Тестовая выборка): ~20-30%. На ней мы проверяем качество.

Ещё есть:

(Val) (валидационная выборка): Оценка модели на итерации обучения.

Необходимые библиотеки: numpy, pandas, scikit-learn

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split

# Данные: 10 объектов, у каждого 2 признака
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10],
              [11, 12], [13, 14], [15, 16], [17, 18], [19, 20]])
# Задача бинарной классификации: классы 0 и 1
y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

# Разбиение данных (80% train, 20% test)
# random_state фиксирует случайность, чтобы результат был воспроизводим
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Размер X_train: {X_train.shape}")
print(f"Размер X_test: {X_test.shape}")

Размер X_train: (8, 2)
Размер X_test: (2, 2)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.DataFrame({
    'feature1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'feature2': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'target': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
})

In [ ]:
# Разделяем на X (признаки) и y (ответы)
X = df.drop('target', axis=1) # Берем всё, кроме колонки с ответами
y = df['target']              # Берем только колонку с ответами

# Разделяем на обучающую (train) и тестовую (test) выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print(f"Строк в обучении: {len(X_train)}")
print(f"Строк в тесте: {len(X_test)}")

Строк в обучении: 8
Строк в тесте: 2


Разделение с сохранением пропорций классов (Stratify)

Крайне важно для задач классификации, если в таргете одного класса намного меньше, чем другого (например, мошенничество 1% и обычные транзакции 99%).

In [ ]:
# Параметр stratify=y гарантирует, что в тесте и трейне будет одинаковый % классов
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

Временное разделение (Time Series)

Если данные — это временной ряд (акции, погода), перемешивать их нельзя (иначе модель будет «заглядывать в будущее»). Нужно просто отрезать кусок из конца.

In [ ]:
# Берем первые 80% строк на обучение, последние 20% на тест
split_index = int(len(df) * 0.8)
train = df.iloc[:split_index]
test = df.iloc[split_index:]

NameError: name 'df' is not defined

Данные из реального мира "грязные". Модели не понимают текст и не любят пропуски.

Основные этапы работы с данными:

Обработка пропусков (Missing Values): Удаление строк с NaN или заполнение их данными:
* Среднее (mean): для симметричных данных без выбросов.
```
imputer = SimpleImputer(strategy='mean')
```
* Медиана (median): для данных с выбросами (устойчива к ним).
```
imputer = SimpleImputer(strategy='median')
```
* Мода (mode): для категориальных признаков.
```
imputer = SimpleImputer(strategy='most_frequent')
```
* Нули/Константы: замена на 0, -999 или "Unknown".
```
# Заполнение value = 0
imputer = SimpleImputer(strategy='constant', fill_value=0)

# Заполнение строкой "Unknown" для категорий
imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
```
* Интерполяция (интерполяция): заполнение промежуточными значениями (линейная, временная).
SimpleImputer не поддерживает интерполяцию. Для этого используется метод Pandas .interpolate()
```
# Линейная интерполяция (только через Pandas)
df['column'] = df['column'].interpolate(method='linear')
```
* Удаление строк с NaN. Это также делается через Pandas, так как импьютеры предназначены для заполнения, а не удаления.
```
# Удалить все строки, где есть хотя бы один NaN
df.dropna(inplace=True)

# Удалить все столбцы, где есть хотя бы один NaN
df.dropna(axis=1, inplace=True)

# Все столбцы, где все значения NaN
df.dropna(axis=1, how='all', inplace=True)

# NaN превышает допустимый порог
# Оставит только те столбцы, где минимум 80% данных заполнены
limit = int(0.8 * len(df))
df.dropna(axis=1, thresh=limit, inplace=True)

# Удаление конкретных столбцов с пропусками
cols_to_check = ['Age', 'Salary']
df.drop(columns=df[cols_to_check].columns[df[cols_to_check].isna().any()], inplace=True)
```

Кодирование категорий (Categorical Encoding): Превращение текста в цифры.

Label Encoding: red=0, green=1, blue=2. (Плохо, если нет порядка).
```
# Для целевой переменной (таргета) или признаков с четким порядком (small < medium < large).
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['target'] = le.fit_transform(df['target'])
```

```
# Ordinal Encoding
# Похоже на Label Encoding, но для признаков и с возможностью задать порядок вручную.

from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
df[['size']] = oe.fit_transform(df[['size']])
```

One-Hot Encoding: Создание новых столбцов-флагов (is_red, is_green...).
```
# Создает столбцы-флаги (0 или 1). Идеально для категорий без порядка
# Через Pandas
df = pd.get_dummies(df, columns=['color'])
# Через Scikit-learn
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
```

Масштабирование (Scaling): Приведение признаков к одному диапазону (например, от 0 до 1 или к нормальному распределению). Это важно для линейных моделей и нейросетей.
```
# Min-Max Scaling (Нормализация)
# Сжимает данные в диапазон от 0 до 1.

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['age', 'salary']] = scaler.fit_transform(df[['age', 'salary']])
```

```
# Standard Scaling (Стандартизация)
# Приводит к среднему = 0 и стандартному отклонению = 1. Устойчивее к выбросам.

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[['age', 'salary']] = scaler.fit_transform(df[['age', 'salary']])
```

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

df = pd.DataFrame({
    'Age': [25, np.nan, 30, 22],
    'Salary': [50000, 60000, np.nan, 35000],
    'City': ['Moscow', 'Paris', 'Moscow', 'London']
})

print("Исходные данные:\n", df)

Исходные данные:
     Age   Salary    City
0  25.0  50000.0  Moscow
1   NaN  60000.0   Paris
2  30.0      NaN  Moscow
3  22.0  35000.0  London


In [ ]:
# Заполнение пропусков (средним значением)
imputer = SimpleImputer(strategy='mean')
df[['Age', 'Salary']] = imputer.fit_transform(df[['Age', 'Salary']])
df

,Age,Salary,City
0,25.000000,50000.000000,Moscow
1,25.666667,60000.000000,Paris
2,30.000000,48333.333333,Moscow
3,22.000000,35000.000000,London


In [ ]:
# Масштабирование (StandardScaler - z-score нормализация)
scaler = StandardScaler()
df[['Age', 'Salary']] = scaler.fit_transform(df[['Age', 'Salary']])
df

,Age,Salary,City
0,-0.233285,0.187317,Moscow
1,0.000000,1.311220,Paris
2,1.516351,0.000000,Moscow
3,-1.283066,-1.498537,London


In [ ]:
# One-Hot Encoding для города
# drop_first=True удаляет первый столбец, чтобы избежать мультиколлинеарности
df = pd.get_dummies(df, columns=['City'], drop_first=True)
df

,Age,Salary,City_Moscow,City_Paris
0,-0.233285,0.187317,True,False
1,0.000000,1.311220,False,True
2,1.516351,0.000000,True,False
3,-1.283066,-1.498537,False,False


# Задача 1.
Создайте массив размера (40, 3) из любых чисел типа int в диапозоне от 10 до 100 в первых двух столбцах (напр., можно использовать генератор списка), в третьем столбце (таргет) числа от типа int 0 или 1.

Далее напишите код, который разбивает датасет на data (признаки) и target (ответы), а затем в соотношении 70% на обучающую и 30% на тестовую выборки. Используйте random_state=10.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

x = np.random.randint(10, 101, size=(40, 2), dtype=int)

y = np.array([np.random.randint(0, 2) for i in range(40)])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=10)

print(f"Размер x_train: {x_train.shape}")
print(f"Размер x_test: {x_test.shape}")

Размер x_train: (28, 2)
Размер x_test: (12, 2)


# Задача 2.
Для каждого из пяти датасетов обработать NaN (и обосновать логику обработки), закодировать таргет (и обосновать логику кодирования).

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer


# Таргет: Купит ли клиент товар
df1 = pd.DataFrame({
    'Age':       [25,     30,   35,     40,    45,     50],
    'ID_System': [np.nan, 102,  np.nan, 105,   np.nan, 107],
    'Target':    ['Yes',  'No', 'No',   'Yes', 'No', 'Yes']
})

x = df1.drop('Target', axis=1)

imputer = SimpleImputer(strategy='constant', fill_value=-1)
df1[['ID_System']] = imputer.fit_transform(df[['ID_System']])

df1

,Age,ID_System,Target
0,25,NaN,Yes
1,30,102.0,No
2,35,NaN,No
3,40,105.0,Yes
4,45,NaN,No
5,50,107.0,Yes


In [ ]:
# Таргет: Уровень подписки (Basic < Silver < Gold — с порядком)
df2 = pd.DataFrame({
    'City': ['Moscow', 'Moscow', 'London', 'Moscow', np.nan, 'Moscow', 'London'],
    'Age': [20, 25, 30, 35, 40, 45, 50],
    'Target': ['Basic', 'Basic', 'Silver', 'Silver', 'Gold', 'Gold', 'Gold']
})

df2

,City,Age,Target
0,Moscow,20,Basic
1,Moscow,25,Basic
2,London,30,Silver
3,Moscow,35,Silver
4,NaN,40,Gold
5,Moscow,45,Gold
6,London,50,Gold


In [ ]:
# Таргет: Группа здоровья (A < B < C — с порядком)
df3 = pd.DataFrame({
    'Pulse': [70, 72, 75, np.nan, 68, 71, 73, 74],
    'Temp': [36.6, 36.7, 36.8, 36.6, 36.9, 36.6, 36.7, 36.8],
    'Target': ['A', 'A', 'B', 'A', 'B', 'A', 'B', 'C']
})

df3

,Pulse,Temp,Target
0,70.0,36.6,A
1,72.0,36.7,A
2,75.0,36.8,B
3,NaN,36.6,A
4,68.0,36.9,B
5,71.0,36.6,A
6,73.0,36.7,B
7,74.0,36.8,C


In [ ]:
# Таргет: Прошел проверку безопасности (Да/Нет)
df4 = pd.DataFrame({
    'Days_Since_Last_Incident': [10, 5, 20, np.nan, 15, 30],
    'Risk_Score': [0.1, 0.2, 0.1, 0.4, 0.2, 0.1],
    'Target': ['Safe', 'Safe', 'Warning', 'Safe', 'Safe', 'Warning']
})

df4

,Days_Since_Last_Incident,Risk_Score,Target
0,10.0,0.1,Safe
1,5.0,0.2,Safe
2,20.0,0.1,Warning
3,NaN,0.4,Safe
4,15.0,0.2,Safe
5,30.0,0.1,Warning


In [ ]:
# Таргет: Кредитный рейтинг (Low < High — с порядком)
df5 = pd.DataFrame({
    'Bonus_Points': [100, 500, np.nan, 200, np.nan, 800],
    'Salary_K': [50, 100, 40, 120, 30, 150],
    'Target': ['Low', 'High', 'Low', 'High', 'Low', 'High']
})

df5

# Задание 3.
Масштабировать признак в датасетах и закодировать таргет

In [ ]:
# MinMaxScaler (Нормализация)
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# таргет: уровень премии (низкий/средний/высокий)
df6 = pd.DataFrame({
    'Completion_Pct': [10, 25, 45, 50, 75, 85, 95, 100],
    'Experience_Years': [1, 2, 3, 4, 5, 6, 7, 8],
    'Target': ['Low', 'Low', 'Medium', 'Medium', 'Medium', 'High', 'High', 'High']
})

In [ ]:
from sklearn.preprocessing import StandardScaler

# таргет: одобрение кредита (да/нет)
df7 = pd.DataFrame({
    'Income_K': [30, 35, 40, 45, 50, 42, 38, 1000],
    'Credit_Score': [600, 620, 640, 610, 650, 630, 615, 800],
    'Target': ['No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes']
})

# Объединение датафреймов
По строкам и собцам

In [ ]:
df8 = pd.DataFrame({
    'City': ['Moscow', 'Moscow', 'London', 'Moscow', np.nan, 'Moscow', 'London'],
    'Age': [20, 25, 30, 35, 40, 45, 50],
    'Target': ['Basic', 'Basic', 'Silver', 'Silver', 'Gold', 'Gold', 'Gold']
})

df9 = pd.DataFrame({
    'City': ['Moscow', 'Moscow', 'London', 'Moscow', np.nan, 'Moscow', 'London'],
    'Age': [20, 25, 30, 35, 40, 45, 50],
    'Target': ['Basic', 'Basic', 'Silver', 'Silver', 'Gold', 'Gold', 'Gold']
})

# axis влияет на конкатенацию: по строкам или столбцам (1, 0)
df10 = pd.concat([df8, df9], axis=1)
df10

,City,Age,Target,City,Age,Target
0,Moscow,20,Basic,Moscow,20,Basic
1,Moscow,25,Basic,Moscow,25,Basic
2,London,30,Silver,London,30,Silver
3,Moscow,35,Silver,Moscow,35,Silver
4,NaN,40,Gold,NaN,40,Gold
5,Moscow,45,Gold,Moscow,45,Gold
6,London,50,Gold,London,50,Gold


In [ ]:
df10 = pd.concat([df8, df9], axis=0)
df10

,City,Age,Target
0,Moscow,20,Basic
1,Moscow,25,Basic
2,London,30,Silver
3,Moscow,35,Silver
4,NaN,40,Gold
5,Moscow,45,Gold
6,London,50,Gold
0,Moscow,20,Basic
1,Moscow,25,Basic
2,London,30,Silver


In [ ]:
# Обратите внимание на индексы строк в прошлой ячейке, когда соединяли по столбцам, поэтому
# Надо передать параметр ignore_index=True
df10 = pd.concat([df8, df9], axis=0, ignore_index=True)
df10

,City,Age,Target
0,Moscow,20,Basic
1,Moscow,25,Basic
2,London,30,Silver
3,Moscow,35,Silver
4,NaN,40,Gold
5,Moscow,45,Gold
6,London,50,Gold
7,Moscow,20,Basic
8,Moscow,25,Basic
9,London,30,Silver


In [ ]:
df8 = pd.DataFrame({
    'City': ['Moscow', 'Moscow', 'London', 'Moscow', np.nan, 'Moscow', 'London'],
    'Age': [20, 25, 30, 35, 40, 45, 50],
    'Gender': ['M', 'M', 'M', 'M', 'M', 'M', 'F'],
    'Target': ['Basic', 'Basic', 'Silver', 'Silver', 'Gold', 'Gold', 'Gold']
})

df9 = pd.DataFrame({
    'City': ['Moscow', 'Moscow', 'London', 'Moscow', np.nan, 'Moscow', 'London'],
    'Age': [20, 25, 30, 35, 40, 45, 50],
    'Target': ['Basic', 'Basic', 'Silver', 'Silver', 'Gold', 'Gold', 'Gold']
})

# параметр join='inner' позволяет конкатенировать датасеты разных рамерностей только по общим признакам
df10 = pd.concat([df8, df9], axis=0, join='inner')
df10

,City,Age,Target
0,Moscow,20,Basic
1,Moscow,25,Basic
2,London,30,Silver
3,Moscow,35,Silver
4,NaN,40,Gold
5,Moscow,45,Gold
6,London,50,Gold
0,Moscow,20,Basic
1,Moscow,25,Basic
2,London,30,Silver
